# Phenomenological Builder


## Modules

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.signal as sg
import pandas as pd
from scipy import interpolate
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec  
import random
import time

## Resample 

In [2]:
def resample(t,h,fs):
    """
    Interpolate array h to the fs sampling frequency.
    Input:
        t  - time array, in seconds
        h  - strain array to be interpolated
        fs - thi is the new sampling frequency
    Output:
        t1 - time array, after resampling
        h1 - new strain array
    """
    # Quick check
    if len(t)!=len(h):
        print("Error: t and h need to have equal sizes")
        return 0
    # Define new time with fs
    t1 = np.arange(t[0],t[-1],1.0/fs)
    # Interpolation
    tck = interpolate.splrep(t,h,s=0)
    h1  = interpolate.splev(t1,tck,der=0)
   
    return t1, h1,fs

## Graphics

In [3]:
def graphs(h,time,Fm,f,tf):
    def add_colorbar(mappable):
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        import matplotlib.pyplot as plt
        last_axes = plt.gca()
        ax = mappable.axes
        fig = ax.figure
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        cbar = fig.colorbar(mappable, cax=cax)
        cbar.set_label('Amplitude String (1/Hz)',fontsize=12)
        plt.sca(last_axes)
        return cbar

    wnd="hamming"
    # if Fm == 200000:
    #   nfft=4*4096
    #   nperseg=4096
    #   noverlap=4096-512
    # elif Fm ==16384:
    r = 1024
    nfft=4*r
    nperseg=r
    noverlap=r-r/8

    fxx, txx, Sxx = sg.spectrogram(h, Fm, window=wnd, nfft=nfft, nperseg=nperseg, noverlap=noverlap, mode='magnitude')

    plt.figure()
    fig,(axs1,axs2) = plt.subplots(1,2,tight_layout = True, figsize=(12,4))
    # plt.suptitle("Spectrogram of the signal (variations of $t_{2}$)",fontsize=16)
    #plt.suptitle("Espectrograma de la señal (variaciones de $t_{2}$)",fontsize=16)
    axs1.plot(time,h,color="darkorange")
    axs1.set_ylabel('$h$')
    axs1.set_xlabel("Time[s]")
    axs1.grid()
    # axs[0,0].text(0.01,500,"$t_{2} = $" + str(t2_1),{'color': 'ivory', 'fontsize': 15})

    z = axs2.pcolormesh(txx, fxx, Sxx,shading="auto",cmap="winter")
    add_colorbar(z)
    #z.set_label('Amplitude String (1/Hz)')
    axs2.plot(tf,f,"r")
    axs2.set_ylabel('Frequency [Hz]')
    axs2.set_xlabel('Time [s]')
    axs2.set_ylim([0,max(f) +100])
    axs2.set_xlim(min(txx),max(txx))

## Phen Implementation

## New "Sigmoid" "Function"

In [4]:
# Define logistic function
def Pseudolog(x,b0,b1):
    lin = b0 + b1*x   
    num = np.exp(lin)
    den = np.exp(lin) + 1
    return 1 - num/den
# Define t
# fs   = 16384
# Ts   = 1/fs
# # Define signal duration
# Tfin = 1.0
# t    = np.arange(0,Tfin,Ts)
# # Create Logistic
# b0 = -34 # Varia de acuerdo a la duracion de la señal h(t)
# b1 = 40 # Queda constante
# W  = Pseudolog(t,b0,b1)

# # Scatter plot of the data and a logistic function
# plt.plot(t, W, ':r' , label='w'  , linewidth=2)
# plt.xlabel("t")
# plt.ylabel("w(t)")
# plt.show()

## Implementación

In [5]:
def Imp0(duration,f_0,f_1,f_driver,Seed):
    Fm    = 200000 #200000
    t_ini = 0.0
    t_2   = duration +0.5
    Q     = 10
    dt    = 1/Fm
    t_s   = np.arange(t_ini,duration,dt)
    f     = np.zeros(len(t_s))    # Set memory to get the frequency
    p_2   = np.zeros(len(t_s))  # Set memory to p_2
    w     = np.zeros(len(t_s))    # Set memory to angular frequency    
    for i in range(0,len(t_s)):
        p_1    =  f_1 - f_0 
        p_3    =  t_2 - t_ini
        ab     = (2*t_2-t_ini-1)*(1-t_ini)
        p_2[i] = t_s[i]-t_ini
        f[i]   = f_0 + 2*p_1*p_3*(p_2[i])/(ab) - p_1*(p_2[i])**2/(ab)
        w[i]   = 2*np.pi*f[i]
    slope = f[(len(t_s) - 1)] - f[0] / duration
    #return t_s,f,w,slope
    # ==============================================

    # t_s,f,nfs = resample(t_s,f,16384)
    # dt = 1/nfs
    # w = 2*np.pi*f
    # ==============================================
    # Delta implementation
    N    = np.arange(t_ini,duration,dt) 
    n    = int(f_driver/(duration-t_ini))
    rng  = np.random.default_rng(seed = Seed)
    t_n1 = rng.uniform(t_ini,duration,n) #Set the uniform distribution of t_n
    t_n  = np.sort(t_n1)       # t_n in progresive order
    t_s  = np.arange(t_ini,duration,dt) #Time in the inicial interval (simple)
    a_n1 = rng.uniform(-1,1,n) 
    a_n1 = a_n1 *t_n
    a    = np.zeros((len(t_s)))
    for i in range(0,len(t_s)):
        for j in range(0,n):
            if t_s[i]-t_n[j]<=0 and t_s[i]-t_n[j]>=-15e-6 :     
                #print("-")
                a[i] = a_n1[j]
                i+=1
    # Model            
    h_t = np.zeros(len(t_s))
    h   = np.zeros(len(t_s))
    # h_t[0] = 0
    # h[0] = 0
    def F(a,h_t,h,w):
        return a - (w/Q)*h_t - (w**2)*h
    for i in range(1,len(t_s)):
        h_t[i] = h_t[i-1] + F(a[i-1],h_t[i-1],h[i-1],w[i-1])*dt
        h[i] = h[i-1] + h_t[i-1]*dt
    h   = h/max(abs(h))


    t_res,h_res,nfs = resample(t_s,h,16384)

    #print(h_res.shape)
    #print(t_res.shape)
    #print(t_s)
    #====================================
    # Graphs
    #graphs(h_res,t_res,nfs,f,t_s) # Esta si va mostrando sin la "sigmoid function"
    #graphs(h,t_s,Fm,f,t_s)
    #====================================
    # Pseudo Sigmoid Function
    t_A = np.linspace(0.0,1.0,len(t_res))
    #Pseudo = PseudoSigFun(t_res)
    Pseudo = Pseudolog(t_A,-34,40)
    #print(Pseudo.shape)
    h_sig = h_res*Pseudo
    #====================================
    # New Graphs
    graphs(h_sig,t_res,nfs,f,t_s)
    return h_res,slope,t_res,f

In [ ]:
# duration = 0.8
# f_0 = 100
# f_1 = 700
# f_driver = 200
# Seed = 8
# Fm    = 200000 #200000
# t_ini = 0.0
# t_2   = duration +0.5
# Q     = 10
# dt    = 1/Fm
# t_s   = np.arange(t_ini,duration,dt)
# f     = np.zeros(len(t_s))    # Set memory to get the frequency
# p_2   = np.zeros(len(t_s))  # Set memory to p_2
# w     = np.zeros(len(t_s))    # Set memory to angular frequency    
# for i in range(0,len(t_s)):
#     p_1    =  f_1 - f_0 
#     p_3    =  t_2 - t_ini
#     ab     = (2*t_2-t_ini-1)*(1-t_ini)
#     p_2[i] = t_s[i]-t_ini
#     f[i]   = f_0 + 2*p_1*p_3*(p_2[i])/(ab) - p_1*(p_2[i])**2/(ab)
#     w[i]   = 2*np.pi*f[i]
# slope = f[(len(t_s) - 1)] - f[0] / duration
# #return t_s,f,w,slope
# # ==============================================

# # t_s,f,nfs = resample(t_s,f,16384)
# # dt = 1/nfs
# # w = 2*np.pi*f
# # ==============================================
# # Delta implementation
# N    = np.arange(t_ini,duration,dt) 
# n    = int(f_driver*(duration-t_ini))
# rng  = np.random.default_rng(seed = Seed)
# t_n1 = rng.uniform(t_ini,duration,n) #Set the uniform distribution of t_n
# t_n  = np.sort(t_n1)       # t_n in progresive order
# t_s1  = np.arange(t_ini,duration,1/16384) #Time in the inicial interval (simple)
# a_n1 = rng.uniform(-1,1,n) 
# a_n1 = a_n1 * t_n
# a    = np.zeros((len(t_s1)))
# cont = 0
# for i in range(0,len(t_s1)):
#     for j in range(0,n):
#         if t_s1[i]-t_n[j]<=0 and t_s1[i]-t_n[j]>=-1e-4 :     
#             #print("-")
#             a[i] = a_n1[j]
#             i+=1
#             cont=j+1
# print(cont)
# print(n)
# #plt.plot(t_n,a_n1,"o")
# plt.plot(t_s1,a,"o")
# plt.show()

# t_s,a,oldfs = resample(t_s1,a,Fm)

# plt.plot(t_s,a,"o")
# plt.show()
# # da.drop_duplicates(subset=[0])
# # print(da)



# # Model            
# h_t = np.zeros(len(t_s))
# h   = np.zeros(len(t_s))
# # h_t[0] = 0
# # h[0] = 0
# def F(a,h_t,h,w):
#     return a - (w/Q)*h_t - (w**2)*h
# for i in range(1,len(t_s)):
#     h_t[i] = h_t[i-1] + F(a[i-1],h_t[i-1],h[i-1],w[i-1])*dt
#     h[i] = h[i-1] + h_t[i-1]*dt
# h   = h/max(abs(h))


# plt.plot(t_s,h)
# plt.show()

In [8]:
def Imp(duration,f_0,f_1,f_driver,Seed):
    Fm    = 200000 #200000
    t_ini = 0.0
    t_2   = duration +0.5
    Q     = 10
    dt    = 1/Fm
    t_s   = np.arange(t_ini,duration,dt)
    f     = np.zeros(len(t_s))    # Set memory to get the frequency
    p_2   = np.zeros(len(t_s))  # Set memory to p_2
    w     = np.zeros(len(t_s))    # Set memory to angular frequency    
    for i in range(0,len(t_s)):
        p_1    =  f_1 - f_0 
        p_3    =  t_2 - t_ini
        ab     = (2*t_2-t_ini-1)*(1-t_ini)
        p_2[i] = t_s[i]-t_ini
        f[i]   = f_0 + 2*p_1*p_3*(p_2[i])/(ab) - p_1*(p_2[i])**2/(ab)
        w[i]   = 2*np.pi*f[i]
    slope = f[(len(t_s) - 1)] - f[0] / duration
    #return t_s,f,w,slope
    # ==============================================

    # t_s,f,nfs = resample(t_s,f,16384)
    # dt = 1/nfs
    # w = 2*np.pi*f
    # ==============================================
    # Delta implementation
    N    = np.arange(t_ini,duration,dt) 
    n    = int(f_driver *(duration-t_ini))
    rng  = np.random.default_rng(seed = Seed)
    t_n1 = rng.uniform(t_ini,duration,n) #Set the uniform distribution of t_n
    t_n  = np.sort(t_n1)       # t_n in progresive order
    dt1 = 1.0/16384
    t_s1  = np.arange(t_ini,duration,dt1) #Time in the inicial interval (simple)
    a_n1 = rng.uniform(-1,1,n) 
    a_n1 = a_n1 * t_n
    a    = np.zeros((len(t_s1)))
    for i in range(0,len(t_s1)):
        for j in range(0,n):
            if t_s1[i]-t_n[j]<=0 and t_s1[i]-t_n[j]>=-1e-4 :     
                #print("-")
                a[i] = a_n1[j]
                i+=1

    t_snew,a,_ = resample(t_s1,a,Fm)


    # Model            
    h_t = np.zeros(len(t_snew))
    h   = np.zeros(len(t_snew))
    # h_t[0] = 0
    # h[0] = 0
    def F(a,h_t,h,w):
        return a - (w/Q)*h_t - (w**2)*h
    for i in range(1,len(t_snew)):
        h_t[i] = h_t[i-1] + F(a[i-1],h_t[i-1],h[i-1],w[i-1])*dt
        h[i] = h[i-1] + h_t[i-1]*dt
    h   = h/max(abs(h))
    # ===================================
    # Resample 
    # t_res = t_s
    # h_res = h
    # nfs = nfs
    t_res,h_res,nfs = resample(t_snew,h,16384)

    #print(h_res.shape)
    #print(t_res.shape)
    #print(t_s)
    #====================================
    # Graphs
    #graphs(h_res,t_res,nfs,f,t_s) # Esta si va mostrando sin la "sigmoid function"
    #graphs(h,t_s,Fm,f,t_s)
    #====================================
    # Pseudo Sigmoid Function
    t_A = np.linspace(0.0,1.0,len(t_res))
    #Pseudo = PseudoSigFun(t_res)
    Pseudo = Pseudolog(t_A,-34,40)
    #print(Pseudo.shape)
    h_sig = h_res*Pseudo
    #====================================
    # New Graphs
    #graphs(h_sig,t_res,nfs,f,t_s)
    return h_res,slope,t_res,f
#    return a, w, Q, t_s, dt,f

## CALL the function

In [ ]:
# Call the function Imp(Duration,f_0,f_1,f_driver,seed)
Dur = 0.8
H,S,Time,F = Imp(Dur,100,700,200,8)

In [ ]:
Dur = 0.8
H,S,Time,F = Imp(Dur,100,700,200,8)

In [ ]:
H,S,Time,F = Imp(Dur,100,700,200,8)

## Generador del dataset

In [ ]:
f_0     = np.linspace(100,400,200) 
f_1     = np.linspace(500,1500,200)
f_drive = np.arange(100,200,1)
Duration= np.linspace(0.3,1.5,200)



#slowly rotating
#slow_Dur = np.linspace(1.0,1.5,500)
slow_f_0 = f_0
slow_f_1 = f_1

#Moderate rotating
moder_Dur = np.linspace(1.0,1.8,500)
moder_f_0 = f_0
moder_f_1 = f_1


#Rapidly rotating
#rapid_Dur = np.linspace(1.0,1.8,500)
rapid_f_0 = f_0
rapid_f_1 = f_1



slow_Dur = np.linspace(0.8,1.2,500)
rapid_Dur= np.linspace(0.3,0.5,500)
moder_Dur= np.linspace(0.5,0.8,500)


for i in range(0,20):
    f=f_0[random.randint(0,199)]
    f1=f_1[random.randint(0,199)]
    H,S,Time,F = Imp(slow_Dur[random.randint(0,499)], f,f1, 200, 8)
    print("Slope",S,"f_0",f,"f_1",f1)

## Slowly Rotating

In [60]:
# Generate a folder to save all the data 
# Desktop/CCSNe_Phen/H9_Slope1800_f0(127.14)_f1(1982.21).csv
import os

mainpath = r'C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator'
try:
    os.chdir(mainpath)
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except FileExistsError:
    print("The directory: {0}",path,"Already exist")
print(os.getcwd())  
# Create a folder to save all the rapidly rotating data
print(os.getcwd())
print(os.getcwd() + "\\Slow\\")


path = os.getcwd() + "\\Slow\\"  #"C:\\Users\\PC\\Desktop\\Tesis\\Fen_tiz\\Images\\"

try:
    os.mkdir(path)
    print("The directory: {0}",path,"Created")
except FileExistsError:
    print("The directory: {0}",path,"Already exist")

C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Slow\
The directory: {0} C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Slow\ Already exist


In [61]:
%%time
#Slowly Slope >1620 - 4990. 
# Primera forma de obtener la data
slow_Dur = np.linspace(0.3,0.7,200)
slow_f_0 = np.linspace(120,130,200)
slow_f_1 = np.linspace(3100,4800,200) #2000
#rapid_Dur = np.linspace(0.35,0.55,200)

cont = 0
for i in range(0,200):
    f=slow_f_0[random.randint(0,199)]
    f1=slow_f_1[random.randint(0,199)]
    H,S,Time,F = Imp(slow_Dur[random.randint(0,199)], f,f1, 400, random.randint(0,200))
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    dat = pd.DataFrame(index=None)
    dat["H"] = H
    dat["Time"] = Time
    dat.to_csv(path + "Slow_run1"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv",index=False)
    print("Slope",S,"f_0",f,"f_1",f1,"Time",Time[-1], f"F_final: {F[-1]}")
    if S>=4990 and S<=1620:
        print("******************************")

print("------------")
h_last,S_last,Time_last,F_last = Imp(0.3,120,3100,400,8)
print("Slope",S_last,"f_0",120,"f_1",3200,"Time",Time_last[-1], f"F_final: {F_last[-1]}")


Slope 2667 f_0 124.52 f_1 3492.96 Time 0.6275634765625 F_final: 2865.816643713487
Slope 2634 f_0 127.99 f_1 3518.59 Time 0.60345458984375 F_final: 2846.4284643163382
Slope 2314 f_0 129.30 f_1 3680.90 Time 0.408447265625 F_final: 2630.5805006088417
Slope 3198 f_0 122.46 f_1 4304.52 Time 0.57330322265625 F_final: 3412.4131108954543
Slope 2659 f_0 124.57 f_1 4629.15 Time 0.305908203125 F_final: 3066.1257433563783
Slope 1912 f_0 128.89 f_1 3142.71 Time 0.39837646484375 F_final: 2236.2868049419535
Slope 2652 f_0 122.36 f_1 3330.65 Time 0.6878662109375 F_final: 2830.052110538801
Slope 2663 f_0 127.69 f_1 3569.85 Time 0.597412109375 F_final: 2877.085742405073
Slope 2621 f_0 126.78 f_1 3296.48 Time 0.68988037109375 F_final: 2805.088596908235
Slope 3110 f_0 127.14 f_1 4082.41 Time 0.61346435546875 F_final: 3318.179909623086
Slope 2933 f_0 125.78 f_1 4236.18 Time 0.4908447265625 F_final: 3189.970660037726
Slope 3363 f_0 122.41 f_1 4475.38 Time 0.5853271484375 F_final: 3573.058396734415
Slope 278

In [ ]:
# Primera forma de obtener la data
slow_Dur = np.linspace(0.1,0.3,200)
slow_f_0 = np.linspace(100,150,200)
slow_f_1 = 2000#np.linspace(1980,2000,200)
#rapid_Dur = np.linspace(0.35,0.55,200)
for i in range(0,10):
    f=slow_f_0[random.randint(0,199)]
    f1=slow_f_1#[random.randint(0,199)]
    H,S,Time,F = Imp0(slow_Dur[random.randint(0,199)], f,f1, 200, 8)
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    dat = pd.DataFrame(index=None)
    dat["H"] = H
    dat["Time"] = Time
    #dat.to_csv("run1"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1,"Time",Time[-1])

In [ ]:
slow_Dur = .5
slow_f_0 = 140
slow_f_1 = 800


L = 10
# Storage
h_storage=[]
keys = []
for i in range(0,L):
    Seed = int(time.time()/1e7 )
    Seed +=i
    #Seed = #Seed
    f=slow_f_0
    f1=slow_f_1
    H,S,Time,F = Imp(slow_Dur, slow_f_0, slow_f_1, 200, Seed)
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    # Storage
    h_storage.append(H)
    keys.append(f"H{i}""_Seed"f"{Seed}")
    #dat = pd.DataFrame()
    #dat["H"] = H
    #dat.to_csv(path + "run2"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1)
    
h = {}
for k in range(0,L):
    h[keys[k]] = h_storage[k]
data = pd.DataFrame(h)
print(data)
data.to_csv(path + "run2"+ "_Slope"f"{Seed}""_f0("f"{f})""_f1("f"{f1}).csv")

## Moderate Rotating

In [62]:
# Generate a folder to save all the data 
# Desktop/CCSNe_Phen/H9_Slope1800_f0(127.14)_f1(1982.21).csv
import os

#mainpath = r'C:\Users\PC\Desktop\CCSNe_Phen'
try:
    os.chdir(mainpath)
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except FileExistsError:
    print("The directory: {0}",path,"Already exist")
print(os.getcwd())  
# Create a folder to save all the rapidly rotating data
print(os.getcwd())
print(os.getcwd() + "\\Moder\\")


path = os.getcwd() + "\\Moder\\"  #"C:\\Users\\PC\\Desktop\\Tesis\\Fen_tiz\\Images\\"

try:
    os.mkdir(path)
    print("The directory: {0}",path,"Created")
except FileExistsError:
    print("The directory: {0}",path,"Already exist")

# mainpath = path
# try:
#     os.chdir(mainpath)
# except FileNotFoundError:
#     print("Directory: {0} does not exist".format(path))
# except FileExistsError:
#     print("The directory: {0}",path,"Already exist")
# print(os.getcwd())  


C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Moder\
The directory: {0} C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Moder\ Already exist


In [63]:
# Moderate 1450 - 1620
moder_Dur = np.linspace(0.8,0.9,200)
moder_f_0 = np.linspace(100,120,200) 
moder_f_1 = np.linspace(1750,1800,200)

for i in range(0,200):
    f=moder_f_0[random.randint(0,199)]
    f1=moder_f_1[random.randint(0,199)]
    H,S,Time,F = Imp(moder_Dur[random.randint(0,199)], f,f1, 400, random.randint(0,199))
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    dat = pd.DataFrame(index = None)
    dat["H"] = H
    dat.to_csv(path + "Moder_run1"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1,"Time:",Time[-1])
    if S>=1620 and S<=1450:
        print("*********************")
h,s,t,f = Imp(1.0,150,1800,400,8)
print(f"Slope: {s}")

Slope 1492 f_0 117.39 f_1 1792.71 Time: 0.81396484375
Slope 1500 f_0 103.72 f_1 1782.91 Time: 0.81549072265625
Slope 1544 f_0 101.31 f_1 1773.37 Time: 0.8662109375
Slope 1510 f_0 107.44 f_1 1772.11 Time: 0.839599609375
Slope 1532 f_0 110.75 f_1 1762.31 Time: 0.87530517578125
Slope 1498 f_0 101.51 f_1 1762.31 Time: 0.82904052734375
Slope 1499 f_0 118.59 f_1 1768.84 Time: 0.8436279296875
Slope 1470 f_0 111.36 f_1 1772.86 Time: 0.80291748046875
Slope 1559 f_0 106.73 f_1 1797.99 Time: 0.86370849609375
Slope 1564 f_0 103.42 f_1 1797.49 Time: 0.86572265625
Slope 1465 f_0 110.25 f_1 1756.03 Time: 0.81195068359375
Slope 1561 f_0 103.62 f_1 1759.55 Time: 0.89990234375
Slope 1474 f_0 114.27 f_1 1772.36 Time: 0.81048583984375
Slope 1543 f_0 108.54 f_1 1753.27 Time: 0.89288330078125
Slope 1555 f_0 102.71 f_1 1762.81 Time: 0.88885498046875
Slope 1457 f_0 113.77 f_1 1754.27 Time: 0.8089599609375
Slope 1553 f_0 108.14 f_1 1795.73 Time: 0.8616943359375
Slope 1474 f_0 110.35 f_1 1752.01 Time: 0.8245239

In [ ]:
moder_Dur = .70
moder_f_0 = 130
moder_f_1 = 1450


L = 10
# Storage
h_storage=[]
keys = []
for i in range(0,L):
    Seed = int(time.time()/1e7 )
    Seed +=i
    #Seed = #Seed
    f=moder_f_0
    f1=moder_f_1
    H,S,Time,F = Imp(moder_Dur, moder_f_0, moder_f_1, 200, Seed)
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    # Storage
    h_storage.append(H)
    keys.append(f"H{i}""_Seed"f"{Seed}")
    #dat = pd.DataFrame()
    #dat["H"] = H
    #dat.to_csv(path + "run2"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1)
    
h = {}
for k in range(0,L):
    h[keys[k]] = h_storage[k]
data = pd.DataFrame(h)
print(data)
data.to_csv(path + "run2"+ "_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")

## Rapidly Rotating

In [64]:
# Generate a folder to save all the data 
# Desktop/CCSNe_Phen/H9_Slope1800_f0(127.14)_f1(1982.21).csv
import os

mainpath = r'C:\Users\PC\Desktop\CCSNe_Phen'
try:
    os.chdir(mainpath)
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except FileExistsError:
    print("The directory: {0}",path,"Already exist")
print(os.getcwd())  

# Create a folder to save all the rapidly rotating data
print(os.getcwd())
print(os.getcwd() + "\\Rapid\\")


path = os.getcwd() + "\\Rapid\\"  #"C:\\Users\\PC\\Desktop\\Tesis\\Fen_tiz\\Images\\"

try:
    os.mkdir(path)
    print("The directory: {0}",path,"Created")
except FileExistsError:
    print("The directory: {0}",path,"Already exist")




Directory: C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Moder\ does not exist
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator
C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Rapid\
The directory: {0} C:\Users\ce_ti\OneDrive\Escritorio\Thesis\Phenom\Generator\Rapid\ Created


In [68]:
# Primera forma de obtener la data
rapid_f_0 = np.linspace(100,150,200)
rapid_f_1 = np.linspace(1100,1500,200)
rapid_Dur = np.linspace(0.6,0.95,200)
#rapid_Dur = np.linspace(0.35,0.55,200)
for i in range(0,200):
    f=rapid_f_0[random.randint(0,199)]
    f1=rapid_f_1[random.randint(0,199)]
    H,S,Time,F = Imp(rapid_Dur[random.randint(0,199)], f,f1, 400, random.randint(0,199))
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    dat = pd.DataFrame(index=None)
    dat["H"] = H
    dat.to_csv(path + "Rapid_run1"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1,"Time:",Time[-1])
    if S<=500 and S>=1450:
        print("*******************")
    

Slope 1041 f_0 145.23 f_1 1317.09 Time: 0.82855224609375
Slope 1204 f_0 142.71 f_1 1425.63 Time: 0.90240478515625
Slope 762 f_0 123.62 f_1 1148.24 Time: 0.62982177734375
Slope 876 f_0 131.41 f_1 1248.74 Time: 0.6790771484375
Slope 906 f_0 141.21 f_1 1126.13 Time: 0.87957763671875
Slope 900 f_0 109.05 f_1 1154.27 Time: 0.78106689453125
Slope 1026 f_0 122.86 f_1 1184.42 Time: 0.9464111328125
Slope 867 f_0 137.94 f_1 1110.05 Time: 0.839111328125
Slope 1073 f_0 103.02 f_1 1393.47 Time: 0.72479248046875
Slope 1045 f_0 131.16 f_1 1411.56 Time: 0.7142333984375
Slope 1181 f_0 105.78 f_1 1327.14 Time: 0.94464111328125
Slope 918 f_0 100.25 f_1 1301.01 Time: 0.6280517578125
Slope 1082 f_0 134.17 f_1 1276.88 Time: 0.91650390625
Slope 1068 f_0 127.39 f_1 1319.10 Time: 0.8355712890625
Slope 1176 f_0 128.39 f_1 1421.61 Time: 0.8531494140625
Slope 1024 f_0 109.30 f_1 1451.76 Time: 0.62451171875
Slope 838 f_0 138.69 f_1 1170.35 Time: 0.7265625
Slope 1209 f_0 122.36 f_1 1403.52 Time: 0.90771484375
Slope

In [ ]:
rapid_f_0 = 120
rapid_f_1 = 1980
rapid_Dur = 0.85


# ==========
#Numero de iteraciones
L = 10
# Storage
h_storage=[]
keys = []
for i in range(0,L):
    Seed = int(time.time())
    f=rapid_f_0
    f1=rapid_f_1
    H,S,Time,F = Imp(rapid_Dur, rapid_f_0,rapid_f_1, 200, Seed)
    S = int(S)
    f = "%.2f" % f
    f1 = "%.2f" % f1
    # Storage
    h_storage.append(H)
    keys.append(f"H{i}""_Seed"f"{Seed}")
    #dat = pd.DataFrame()
    #dat["H"] = H
    #dat.to_csv(path + "run2"+f'H{i}'"_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")
    print("Slope",S,"f_0",f,"f_1",f1)
    
h = {}
for k in range(0,L):
    h[keys[k]] = h_storage[k]
data = pd.DataFrame(h)
print(data)
data.to_csv(path + "run2"+ "_Slope"f"{S}""_f0("f"{f})""_f1("f"{f1}).csv")

In [ ]:
# Call the function Imp(Duration,f_0,f_1,f_driver,seed)
for i in range(0,10):
  H,S,Time,F = Imp(Duration[random.randint(0,199)], f_0[random.randint(0,199)], f_1[random.randint(0,199)], 200,int(time.time()))
  print(Duration[random.randint(0,199)], f_0[random.randint(0,199)], f_1[random.randint(0,199)], 200,int(time.time()))
  print("Slope",S)

In [ ]:
# 0.993467336683417 197.98994974874373 1059.2964824120604 200 1673543024
# 0.9452261306532663 127.1356783919598 890.4522613065327 200 1673543031
# 0.8547738693467337 121.10552763819095 721.608040201005 200 1673543038
# 0.3422110552763819 291.4572864321608 732.1608040201005 200 1673543045
# 0.728140703517588 389.44723618090455 967.8391959798995 200 1673543053
# 1.071859296482412 348.7437185929648 630.1507537688442 200 1673543062
# 1.150251256281407 113.5678391959799 528.1407035175879 200 1673543069
# 1.264824120603015 252.26130653266333 1192.964824120603 200 1673543076
# 0.6798994974874372 110.55276381909547 1024.1206030150754 200 1673543083
# 0.921105527638191 205.52763819095478 742.713567839196 200 1673543090



H,S,Time,F = Imp(0.993467336683417,197.98994974874373,1059.2964824120604,200,1673543024)

In [ ]:
# from matplotlib.axis import YAxis
# ## Leer e interpretar datos de json



# Noise = pd.read_json("Noise_H1.json")

# print(Noise)



# X = Noise["segments"]
# Y = Noise["start"]
# Z = Noise["end"]
# print(X[0])



# plt.plot(X)

# for i in range(0,413):


## Otras cosas


In [ ]:
# Try to made a sigmoid function

print(Time.shape)

Shape = len(Time)
Partition = int(Shape/8)
print(type(Partition))
L = np.linspace(0.38,1.0,Partition)
Log = - np.log(L)
F = np.ones(Shape - Partition)
D = np.concatenate((F,Log),axis=None)
print(Log)
print(D)
plt.plot(Time,D)
8192/8

In [ ]:
print(S)
plt.plot(Time,H,color="blue")
plt.plot(Time,H*D,color="green")
# Export to json spectrogram information

In [ ]:
#f 
#log= np.log(Time)
#f = np.log(1 - (1/Time))
#f = 1/(1 + np.exp(Time))**2
#f = 1/(1 + np.exp(-Time))
K = 1#/Time
epsT = np.exp(-K* Time)
#f = 2/(1.0 + np.exp(Time))

#f =1 + .21/log
#f =1 + Time/log
#f = 1 - (1/(1+epsT))
f = 2 - 1/(epsT)
#f =1 - (1/((1 + epsT)**5))
print(f)
#print(log)
#plt.plot(Time,H)
plt.plot(Time,f,color="red")
plt.plot(Time,H*f,color="green")
plt.show()